In [1]:
import sys 
sys.path.append('../..')

In [2]:
from modules.hh_dates import create_bus_dates

In [3]:
date_index = create_bus_dates('2017-01-01', '2019-03-31', 'week')
date_index

DatetimeIndex(['2017-01-06', '2017-01-13', '2017-01-20', '2017-01-27',
               '2017-02-03', '2017-02-10', '2017-02-17', '2017-02-24',
               '2017-03-03', '2017-03-10',
               ...
               '2019-01-25', '2019-02-01', '2019-02-08', '2019-02-15',
               '2019-02-22', '2019-03-01', '2019-03-08', '2019-03-15',
               '2019-03-22', '2019-03-29'],
              dtype='datetime64[ns]', length=117, freq='W-FRI')